In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.tools.checkpoint_load import load_model
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")


INFO     Set logging level to info


In [2]:
batch_size = 512
model_name = "toy_convnet"
dataset_name = "cifar10"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)

# LAB1_CUSTOM_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/lab-1_jsc-custom/software/training_ckpts/best.ckpt"
# model = load_model(load_name=LAB1_CUSTOM_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
    max_batches=1
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [3]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'conv2d':
        print(node.name)
        print(50*'-')
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # print(node.meta['mase'].parameters['common']['args']['weight']['value'])
        # print(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        pprint(mg.modules[node.target].weight)
        pprint(mg.modules[node.target].state_dict())
        print(50*'-')

block_1_0
--------------------------------------------------
Parameter containing:
tensor([[[[ 0.0511,  0.0415, -0.1785],
          [-0.1422,  0.0397,  0.0046],
          [-0.0025, -0.1389,  0.1855]],

         [[ 0.0603, -0.1280, -0.1330],
          [-0.1860, -0.1275,  0.1040],
          [-0.0130, -0.0236,  0.1780]],

         [[ 0.1138,  0.1514,  0.0914],
          [ 0.1328, -0.0420,  0.0968],
          [ 0.0064, -0.0444,  0.0383]]],


        [[[ 0.1165,  0.1459,  0.0299],
          [ 0.1262, -0.1186, -0.1840],
          [ 0.1063, -0.1344,  0.0199]],

         [[ 0.1267,  0.1723,  0.0052],
          [ 0.0046,  0.1300, -0.1681],
          [-0.0056,  0.0401,  0.0087]],

         [[ 0.1411, -0.0832,  0.0250],
          [-0.1240, -0.0799,  0.1830],
          [-0.0106, -0.0095, -0.0477]]],


        [[[ 0.0983,  0.0470,  0.1404],
          [ 0.0080,  0.0048, -0.0296],
          [ 0.1610,  0.0749, -0.1594]],

         [[ 0.1525, -0.1495, -0.1402],
          [ 0.0065,  0.0654, -0.1743],
  

In [4]:
from chop.passes.graph.transforms import (
    prune_transform_pass,
)
pass_args = {
    "weight":{
        "scope" : "local",
        "granularity" : "channel",
        "method" :  "l1-norm",
        "sparsity" : 0.5,
    },
    "activation":{
        "scope" : "local",
        "granularity" : "elementwise",
        "method" : "l1-norm",
        "sparsity" : 0.5,
    },
}
 
mg, _ = prune_transform_pass(mg, pass_args)



4
4
3
3
2


In [13]:
model = mg.model
# print(list(model.parameters()))
# for name, param in model.named_parameters():
#     print(f"Module: {name}")
#     if hasattr(param, 'parametrizations'):
#         for param_name, parametrization in param.parametrizations.items():
#             mask = parametrization.mask
#             print(f"Parametrization: {param_name}")
#             print(f"Mask: {mask}")
#             print(50*'-')
#     else:
#         print("No parametrizations found.")
#         print(50*'-')

# for name, param in model.named_parameters():
#     print(f"Module: {name}")
#     # print(param)
#     print(50*'-')

for name, param in model.named_buffers():
    print(f"Module: {name}")
    print(param)
    print(50*'-')


Module: block_1.0.parametrizations.weight.0.mask
tensor([[[[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]]],


        [[[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]],

         [[False, False, False],
          [False, False, False],
          [False, False, False]]],


        [[[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],

         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]]],


        [[[False, False, False],
   

In [ ]:
from pprint import pprint

from chop.passes.graph.utils import get_mase_op

# pprint(mg.meta['mase'].__dict__)

for node in mg.fx_graph.nodes:
    if get_mase_op(node) == 'conv2d':
        print(f"Layer: {node.name}")
        # pprint(node.meta['mase'].parameters['common'])
        # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
        # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
        pprint(mg.modules[node.target].data_out_0)
        #pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
        pprint(mg.modules[node.target].state_dict())
        # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])
        #print(mg.modules[node.target].parametrizations['weight'][0].mask == mg.modules[node.target].parametrizations['weight'][1].mask)
        total_w = 0
        pruned_w = 0
        w = mg.modules[node.target].weight
        for s in w:
            total_w += s.numel()
            pruned_w += s.numel() - s.nonzero().numel()

        pruned_percent = pruned_w / total_w
        print(f"Pruned percent: {pruned_percent}")

        print(50*'-')


In [ ]:
# from pprint import pprint

# from chop.passes.graph.utils import get_mase_op

# # pprint(mg.meta['mase'].__dict__)

# for node in mg.fx_graph.nodes:
#     if get_mase_op(node) == 'linear':
#         print(f"Layer: {node.name}")
#         # pprint(node.meta['mase'].parameters['common'])
#         # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
#         # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
#         pprint(mg.modules[node.target].weight)
#         # pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
#         # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

#         total_w = 0
#         pruned_w = 0
#         mask_2= mg.modules[node.target].parametrizations['weight'][0].mask
#         for s in mask_2:
#             total_w += s.numel()
#             pruned_w += s.numel() - s.nonzero().numel()

#         pruned_percent = pruned_w / total_w
#         print(f"Pruned percent: {pruned_percent}")

#         print(50*'-')


In [ ]:
# from chop.actions import train
# import torch

# # print(isinstance(mg.model, torch.nn.Module))

# model = mg.model
# model_info = get_model_info('jsc-tiny')
# dataset_info = get_dataset_info('jsc')
# task = "cls"

# train_params = {
#     "model": model,
#     "model_info": model_info,
#     "data_module": data_module,
#     "dataset_info": dataset_info,
#     "task": task,
#     "optimizer": "adam",
#     "learning_rate": 1e-3,
#     "weight_decay": 0,
#     "plt_trainer_args": {
#         "max_epochs": 1,
#     }, 
#     "auto_requeue": False,
#     "save_path": None,
#     "visualizer": None,
#     "load_name": None,
#     "load_type": None
# }

# train(**train_params)

In [ ]:
# from pprint import pprint

# from chop.passes.graph.utils import get_mase_op


# # pprint(mg.meta['mase'].__dict__)

# for node in mg.fx_graph.nodes:
#     if get_mase_op(node) == 'linear':
#         print(node.name)
#         print(50*'-')
#         # pprint(node.meta['mase'].parameters['common'])
#         # pprint(node.meta['mase'].parameters['common']['args']['data_in_0']['value'])
#         # pprint(node.meta['mase'].parameters['common']['args']['weight']['value'])
#         pprint(mg.modules[node.target].weight)
#         pprint(mg.modules[node.target].parametrizations['weight'][0].mask)
#         # pprint(node.meta['mase'].parameters['common']['results']['data_out_0']['value'])

#         print(50*'-')